<a href="https://colab.research.google.com/github/OlivierDataS/github_actions/blob/master/Projet_Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de l'environnement - la première fois seulement !

In [ ]:
# For Python
pip install pandas

# ORM for databases
pip install sqlalchemy # ORM for databases

#Python SQL Toolkit - SqlAlchemy installation
pip install ipython-sql

Activation de SqlMagic :

In [ ]:
%load_ext sql
#%reload_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%config SqlMagic.autocommit=True

Connection/creation à la BD/schema :

In [ ]:
%sql sqlite:///Itineraires.db

'Connected: @Itineraires.db'

Load tools for Python :

In [ ]:
import pandas as pd
from google.colab import files
from google.colab import drive
import io
#from sqlalchemy import create_engine, ForeignKey
#from sqlalchemy import Column, Date, Integer, String
#from sqlalchemy.ext.declarative import declarative_base
import sqlite3
conn = sqlite3.connect('Itineraires.db')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#engine = create_engine("sqlite:///Itineraires.db")
#connection = engine.raw_connection()

=================================== Chargement des données des référentiels

Création de la table référentielle des villes :

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS city_repository (
city_id INTEGER PRIMARY KEY AUTOINCREMENT,
nom_commune VARCHAR,
code_postal INTEGER NOT NULL,
Lieu_dit VARCHAR,
gps_latitude FLOAT,
gps_longitude FLOAT,
Date_maj TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

 * sqlite:///Itineraires.db


[]

Création de la table référentielle des catégories par hierarchies :

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS activity_hierarchy (
activity_id INTEGER NOT NULL,
activity_name VARCHAR,
activity_to INTEGER, 
Date_maj TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

 * sqlite:///Itineraires.db


[]

Import des données "city" par l'intermédiaire d'un fichier puis d'un dataframe (correspondance des noms de colonnes) :

In [ ]:
import_city_repository = pd.read_csv('/content/drive/MyDrive/CSV4DB/city_repository.csv')

In [ ]:
import_city_repository.to_sql('city_repository', conn, if_exists='append', index=False)

Vérifications :

In [ ]:
import_city_repository.count()

nom_commune      85
code_postal      85
Lieu_dit         61
gps_latitude     85
gps_longitude    85
dtype: int64

In [ ]:
%%sql
SELECT COUNT(*) FROM city_repository

 * sqlite:///Itineraires.db


COUNT(*)
85


In [ ]:
%%sql
SELECT * FROM city_repository
LIMIT 5

 * sqlite:///Itineraires.db


city_id,nom_commune,code_postal,Lieu_dit,gps_latitude,gps_longitude,Date_maj
1,ST DENIS,97490,BOIS DE NEFLES ST DENIS,-20.933029099,55.446977555,2023-02-11 18:18:27
2,ST DENIS,97490,ST DENIS CHAUDRON,-20.933029099,55.446977555,2023-02-11 18:18:27
3,ST DENIS,97490,LA BRETAGNE,-20.933029099,55.446977555,2023-02-11 18:18:27
4,ST DENIS,97490,MOUFIA,-20.933029099,55.446977555,2023-02-11 18:18:27
5,ST DENIS,97490,STE CLOTILDE,-20.933029099,55.446977555,2023-02-11 18:18:27


Import des données "activity" :

In [ ]:
import_activity_hierarchy = pd.read_csv('/content/drive/MyDrive/CSV4DB/activity_hierarchy.csv')

In [ ]:
import_activity_hierarchy.to_sql('activity_hierarchy', conn, if_exists='append', index=False)

Vérifications :

In [ ]:
import_activity_hierarchy.count()

activity_id      252
activity_name    252
activity_to      251
dtype: int64

In [ ]:

%%sql
SELECT COUNT(*) FROM activity_hierarchy

 * sqlite:///Itineraires.db


COUNT(*)
252


In [ ]:
%%sql
SELECT * FROM activity_hierarchy
LIMIT 5

 * sqlite:///Itineraires.db


activity_id,activity_name,activity_to,Date_maj
1,PointOfInterest,None,2023-02-11 18:37:45
2,PlaceOfInterest,1,2023-02-11 18:37:45
3,ActivityProvider,2,2023-02-11 18:37:45
4,CulturalSite,2,2023-02-11 18:37:45
5,FoodEstablishment,2,2023-02-11 18:37:45


===================================
Chargement des données POI - les tables sont crées depuis leurs dataframes.

Dataframe/table du fichier reunion_clean.csv :

In [ ]:
poi_reunion_clean = pd.read_csv('/content/drive/MyDrive/CSV4DB/reunion_clean.csv')

In [ ]:
%sql persist poi_reunion_clean

 * sqlite:///Itineraires.db


'Persisted poi_reunion_clean'

Vérifications :

In [ ]:
poi_reunion_clean.count()

ID                     1750
Nom_du_POI             1750
Latitude               1750
Longitude              1750
Adresse_postale        1655
Date_de_mise_a_jour    1750
Contacts_du_POI        1655
Description              29
region                 1750
commune                1750
code_postal            1750
category_0             1743
category_1             1224
category_2               78
dtype: int64

In [ ]:
%%sql
SELECT count(*) FROM poi_reunion_clean;

 * sqlite:///Itineraires.db


count(*)
1750


In [ ]:
%%sql
SELECT * FROM poi_reunion_clean
LIMIT 5;

 * sqlite:///Itineraires.db


index,ID,Nom_du_POI,Latitude,Longitude,Adresse_postale,Date_de_mise_a_jour,Contacts_du_POI,Description,region,commune,code_postal,category_0,category_1,category_2
0,187a27ff-8ad2-3298-9b2f-064023dd042b,Lentille des Gourmets (La),-21.13588418,55.479501,"40 E, chemin des Trois Mares",2022-05-30,Lentille des Gourmets (La)#+33 2 62 31 85 85#hotel@lecilaos.re#http://leschenets-lecilaos.re/,None,54,Cilaos,97413,FoodEstablishment,Restaurant,None
1,ac5515a8-d78e-35ed-a51e-6fc6b1ddbb2b,Neptune (Le),-21.34144,55.462664,"117, boulevard Hubert Delisle",2021-07-13,Neptune (Le)#+33 2 62 61 61 61#reservation@lebattantdeslames.com#http://www.lebattantdeslames.com/,None,54,Saint-Pierre,97410,ActivityProvider,LeisureSportActivityProvider,None
2,43079596-c868-3cfe-a280-aeaf15d5666c,Grand Large (Le),-21.34144,55.462664,"117, boulevard Hubert Delisle",2021-07-13,Grand Large (Le)#+33 2 62 61 61 61#reservation@lebattantdeslames.com#http://www.lebattantdeslames.com/,None,54,Saint-Pierre,97410,FoodEstablishment,Restaurant,None
3,f2a140d5-152c-300b-accb-67619012a89a,Concorde (Le),-20.89625806,55.44723605,"91 bis, allée des Topazes - Bellepierre",2019-10-10,Concorde (Le)#+33 2 62 51 51 51#info@hotel-bellepierre.com#http://www.hotel-bellepierre.com/,None,54,Saint-Denis,97400,FoodEstablishment,Restaurant,ActivityProvider
4,9c77eb5d-8fd9-3700-8e77-e830870e9669,Jardin de la Maison d'Edith (Le),-20.889919,55.407147,"59, chemin Commins",2020-05-07,Jardin de la Maison d'Edith (Le)#+33 6 92 69 66 05#maisondedith@gmail.com#http://www.maisondedith.com/,None,54,Saint-Denis,97417,CulturalSite,None,None


Dataframe/table du fichier reunion_poi_2.csv :


In [ ]:
poi_reunion_2 = pd.read_csv('/content/drive/MyDrive/CSV4DB/reunion_poi_2.csv')

In [ ]:
%sql persist poi_reunion_2

 * sqlite:///Itineraires.db


'Persisted poi_reunion_2'

Vérifications :

In [ ]:
poi_reunion_2.count()

ID                     314
Nom_du_POI             314
Latitude               314
Longitude              314
Adresse_postale        227
Date_de_mise_a_jour    314
Contacts_du_POI        227
Description             28
region                 314
commune                314
code_postal            314
category_0             307
category_1             105
category_2              21
dtype: int64

In [ ]:
%%sql
SELECT count(*) FROM poi_reunion_2;

 * sqlite:///Itineraires.db


count(*)
314


Dataframe/table du fichier reunion_activity.csv :

In [ ]:
poi_reunion_activity = pd.read_csv('/content/drive/MyDrive/CSV4DB/reunion_activity.csv')

In [ ]:
%sql persist poi_reunion_activity


 * sqlite:///Itineraires.db


'Persisted poi_reunion_activity'

Vérifications :

In [ ]:
poi_reunion_activity.count()

ID                     241
Nom_du_POI             241
Latitude               241
Longitude              241
Adresse_postale        233
Date_de_mise_a_jour    241
Contacts_du_POI        233
Description              1
region                 241
commune                241
code_postal            241
category_0             241
category_1             241
category_2               2
dtype: int64

In [ ]:
%%sql
SELECT COUNT(*) FROM poi_reunion_activity

 * sqlite:///Itineraires.db


COUNT(*)
241


Dataframe/table du fichier reunion_coordonnees.csv :

In [ ]:
poi_reunion_coordonnees = pd.read_csv('/content/drive/MyDrive/CSV4DB/reunion_coordonnees.csv')

In [ ]:
%%sql
persist poi_reunion_coordonnees

 * sqlite:///Itineraires.db


'Persisted poi_reunion_coordonnees'

Vérifications :

In [ ]:
poi_reunion_coordonnees.count()

ID            1750
Nom_du_POI    1750
Latitude      1750
Longitude     1750
dtype: int64

In [ ]:
%%sql
SELECT COUNT(*) FROM poi_reunion_coordonnees;

 * sqlite:///Itineraires.db


COUNT(*)
1750


Dataframe/table du fichier reunion_food.csv :

In [ ]:
poi_reunion_food = pd.read_csv('/content/drive/MyDrive/CSV4DB/reunion_food.csv')

In [ ]:
%%sql
persist poi_reunion_food

 * sqlite:///Itineraires.db


'Persisted poi_reunion_food'

Vérifications

In [ ]:
poi_reunion_food.count()

ID                     1195
Nom_du_POI             1195
Latitude               1195
Longitude              1195
Adresse_postale        1195
Date_de_mise_a_jour    1195
Contacts_du_POI        1195
Description               0
region                 1195
commune                1195
code_postal            1195
category_0             1195
category_1              878
category_2               55
dtype: int64

In [ ]:
%%sql
SELECT COUNT(*) FROM poi_reunion_food;

 * sqlite:///Itineraires.db


COUNT(*)
1195


Dataframe/table du fichier reunion_ville_cp.csv :

In [ ]:
poi_reunion_ville_cp = pd.read_csv('/content/drive/MyDrive/CSV4DB/reunion_ville_cp.csv')

In [ ]:
%%sql
persist poi_reunion_ville_cp

 * sqlite:///Itineraires.db


'Persisted poi_reunion_ville_cp'

Vérifications :

In [ ]:
poi_reunion_ville_cp.count()

code_postal    37
commune        37
region         37
dtype: int64

In [ ]:
%%sql
SELECT COUNT(*) FROM poi_reunion_ville_cp

 * sqlite:///Itineraires.db


COUNT(*)
37


Dataframe/table du fichier ontology_hierarchy_descending (2).csv :

In [ ]:
ontology_hierarchy_descending = pd.read_csv('/content/drive/MyDrive/CSV4DB/ontology_hierarchy_descending (2).csv')

In [ ]:
%sql persist ontology_hierarchy_descending

 * sqlite:///Itineraires.db


'Persisted ontology_hierarchy_descending'

Vérifications :

In [ ]:
ontology_hierarchy_descending.count()

PointOfInterest    231
PlaceOfInterest    231
activity0          231
activity1          231
activity2           62
dtype: int64

In [ ]:
%%sql
SELECT count(*) FROM ontology_hierarchy_descending;

 * sqlite:///Itineraires.db


count(*)
231


Dataframe/table du fichier subclass_class_relations.csv :

In [ ]:
activity_subclass_class_relations = pd.read_csv('/content/drive/MyDrive/CSV4DB/subclass_class_relations.csv')

In [ ]:
%sql persist activity_subclass_class_relations

 * sqlite:///Itineraires.db


'Persisted activity_subclass_class_relations'


Vérifications :

In [ ]:
activity_subclass_class_relations.count()

subclass    250
class       250
dtype: int64

In [ ]:
%%sql
SELECT count(*) FROM activity_subclass_class_relations;

 * sqlite:///Itineraires.db


count(*)
250


========================================================================
Présence des tables : 

*   city_repository 
*   activity_hierarchy
*   poi_reunion_clean
*   poi_reunion_2.
*   poi_reunion_activity
*   poi_reunion_coordonnees
*   poi_reunion_food
*   poi_reunion_ville_cp
*   ontology_hierarchy_descending
*   activity_subclass_class_relations



Exemples de requêtes :

In [ ]:
%%sql
SELECT nom_commune, COUNT(Lieu_dit), gps_latitude, gps_longitude
FROM city_repository
GROUP BY nom_commune
ORDER BY 2 DESC
LIMIT 5

 * sqlite:///Itineraires.db


nom_commune,COUNT(Lieu_dit),gps_latitude,gps_longitude
ST DENIS,12,-20.933029099,55.446977555
ST PAUL,10,-21.044520611,55.322330771
ST PIERRE,6,-21.312265663,55.49381116
LE TAMPON,5,-21.223167157,55.558552873
STE MARIE,3,-20.946751887,55.530655328


In [ ]:
%%sql
SELECT * FROM ontology_hierarchy_descending
LIMIT 5

 * sqlite:///Itineraires.db


index,PointOfInterest,PlaceOfInterest,activity0,activity1,activity2
0,PointOfInterest,PlaceOfInterest,ActivityProvider,CulturalActivityProvider,None
1,PointOfInterest,PlaceOfInterest,ActivityProvider,LeisureSportActivityProvider,None
2,PointOfInterest,PlaceOfInterest,CulturalSite,ArcheologicalSite,MegalithDolmenMenhir
3,PointOfInterest,PlaceOfInterest,CulturalSite,ArcheologicalSite,RomanPath
4,PointOfInterest,PlaceOfInterest,CulturalSite,ArcheologicalSite,Ruins


In [ ]:
%%sql
SELECT class, subclass  
FROM activity_subclass_class_relations
LIMIT 5

 * sqlite:///Itineraires.db


class,subclass
ReligiousSite,Abbey
PlaceOfInterest,ActivityProvider
Transport,Airfield
Transport,Airport
NaturalHeritage,AlpinePasture


In [ ]:
%%sql
SELECT activity_name 
FROM activity_hierarchy
WHERE activity_to =
(SELECT activity_id 
FROM activity_hierarchy
WHERE activity_name = 'CulturalSite')


 * sqlite:///Itineraires.db


activity_name
ArcheologicalSite
CityHeritage
DefenceSite
InterpretationCentre
Library
Museum
ParkAndGarden
ReligiousSite
RemarkableBuilding
RemembranceSite


In [ ]:
%%sql
SELECT category_1, COUNT(category_1)
FROM poi_reunion_clean
WHERE category_0 = 'CulturalSite'
GROUP BY category_1
LIMIT 5;

 * sqlite:///Itineraires.db


category_1,COUNT(category_1)
None,0
ReligiousSite,9
RemarkableBuilding,1
TechnicalHeritage,33


In [ ]:
%%sql
SELECT Nom_du_POI, commune, code_postal, Latitude, Longitude, category_1
FROM poi_reunion_clean
WHERE category_1 IN
(
  SELECT activity_name 
FROM activity_hierarchy
WHERE activity_to =
(SELECT activity_id 
FROM activity_hierarchy
WHERE activity_name = 'CulturalSite')
)
ORDER BY 3;

 * sqlite:///Itineraires.db


Nom_du_POI,commune,code_postal,Latitude,Longitude,category_1
Temple Kali Kampal,Saint-Denis,97400,-20.880661,55.458077,ReligiousSite
Cathédrale Saint-Denis,Saint-Denis,97400,-20.877163,55.448789,ReligiousSite
Mosquée Noor-e-Islam,Saint-Denis,97400,-20.879185,55.452474,ReligiousSite
Saga du Rhum (La),Saint-Pierre,97410,-21.312109,55.465557,TechnicalHeritage
Rhums et Punchs Isautier,Saint-Pierre,97410,-21.320276,55.474025,TechnicalHeritage
Charrette de Baster (La),Saint-Pierre,97410,-21.325298,55.484798,TechnicalHeritage
Coopérative Provanille (La),Bras-Panon,97412,-21.000724,55.687307,TechnicalHeritage
Chai de Cilaos (Le),Cilaos,97413,-21.134936,55.47241,TechnicalHeritage
Edena,La Possession,97419,-20.962967,55.321357,TechnicalHeritage
Atelier Mascarin,Le Port,97420,-20.949323,55.288668,TechnicalHeritage


In [ ]:
%%sql
SELECT prc.Nom_du_POI, prvc.commune, prvc.code_postal, prc.Latitude "POI Lat", prc.Longitude "POI 'Long", prc.category_1
FROM poi_reunion_clean prc, poi_reunion_ville_cp prvc
WHERE prc.code_postal = prvc.code_postal
AND category_1 IN
(
  SELECT activity_name 
FROM activity_hierarchy
WHERE activity_to =
(SELECT activity_id 
FROM activity_hierarchy
WHERE activity_name = 'CulturalSite')
)
ORDER BY 3
LIMIT 5;

 * sqlite:///Itineraires.db


Nom_du_POI,commune,code_postal,POI Lat,POI 'Long,category_1
Temple Kali Kampal,Saint-Denis,97400,-20.880661,55.458077,ReligiousSite
Cathédrale Saint-Denis,Saint-Denis,97400,-20.877163,55.448789,ReligiousSite
Mosquée Noor-e-Islam,Saint-Denis,97400,-20.879185,55.452474,ReligiousSite
Saga du Rhum (La),Saint-Pierre,97410,-21.312109,55.465557,TechnicalHeritage
Rhums et Punchs Isautier,Saint-Pierre,97410,-21.320276,55.474025,TechnicalHeritage


In [ ]:
%%sql
SELECT *
FROM city_repository
LIMIT 10;

 * sqlite:///Itineraires.db


city_id,nom_commune,code_postal,Lieu_dit,gps_latitude,gps_longitude,Date_maj
1,ST DENIS,97490,BOIS DE NEFLES ST DENIS,-20.933029099,55.446977555,2023-02-11 18:18:27
2,ST DENIS,97490,ST DENIS CHAUDRON,-20.933029099,55.446977555,2023-02-11 18:18:27
3,ST DENIS,97490,LA BRETAGNE,-20.933029099,55.446977555,2023-02-11 18:18:27
4,ST DENIS,97490,MOUFIA,-20.933029099,55.446977555,2023-02-11 18:18:27
5,ST DENIS,97490,STE CLOTILDE,-20.933029099,55.446977555,2023-02-11 18:18:27
6,ST JOSEPH,97480,LES LIANES,-21.306243323,55.641977434,2023-02-11 18:18:27
7,ST JOSEPH,97480,VINCENDO,-21.306243323,55.641977434,2023-02-11 18:18:27
8,ST JOSEPH,97480,None,-21.306243323,55.641977434,2023-02-11 18:18:27
9,ST JOSEPH,97480,JEAN PETIT,-21.306243323,55.641977434,2023-02-11 18:18:27
10,ST BENOIT,97470,None,-21.091842698,55.649276452,2023-02-11 18:18:27


In [ ]:
%%sql
SELECT Nom_du_POI, Adresse_postale, commune, Description, category_0, category_1, category_2
FROM poi_reunion_activity 
WHERE category_1 ='LeisureSportActivityProvider'
LIMIT 5;


 * sqlite:///Itineraires.db


Nom_du_POI,Adresse_postale,commune,Description,category_0,category_1,category_2
Neptune (Le),"117, boulevard Hubert Delisle",Saint-Pierre,None,ActivityProvider,LeisureSportActivityProvider,None
Zen (Le),"2, rue Doret",Saint-Denis,None,ActivityProvider,LeisureSportActivityProvider,None
Varangue (La),"14, rue du Stade - Montgaillard",Saint-Denis,None,ActivityProvider,LeisureSportActivityProvider,None
Casino du Sud,"42, boulevard Hubert Delisle",Saint-Pierre,None,ActivityProvider,LeisureSportActivityProvider,None
Théâtre Vladimir Canter,15 avenue René Cassin - Campus Universitaire du Moufia,Saint-Denis,None,ActivityProvider,LeisureSportActivityProvider,None


In [ ]:
%%sql
SELECT category_1, count(category_1) 
FROM poi_reunion_2 
GROUP BY category_1
LIMIT 5;

 * sqlite:///Itineraires.db


category_1,count(category_1)
None,0
Coastline,3
Mountain,7
NaturalHeritage,18
PointOfView,13


In [ ]:
%%sql
SELECT category_2, count(category_2) 
FROM poi_reunion_2  
GROUP BY category_2
LIMIT 5;

 * sqlite:///Itineraires.db


category_2,count(category_2)
None,0
Bridge,1
Church,5
Mosque,1
Source,11


In [ ]:
%%sql
SELECT Nom_du_POI, Adresse_postale 
FROM poi_reunion_2
WHERE category_0 = 'Beach' 
AND commune = 'Saint-Leu';

 * sqlite:///Itineraires.db


Nom_du_POI,Adresse_postale
Plage Citerne 46,None
Plage de Saint-Leu,None


In [ ]:
%%sql
SELECT commune, count(commune) as nb_poi 
FROM poi_reunion_clean
GROUP BY commune
ORDER BY nb_poi DESC
LIMIT 10;

 * sqlite:///Itineraires.db


commune,nb_poi
Saint-Paul,370
Saint-Denis,302
Saint-Pierre,283
Saint-Leu,100
Saint-André,73
Le Port,63
Le Tampon,57
Saint-Benoît,51
Sainte-Marie,47
Saint-Louis,44


In [ ]:
%%sql
SELECT * 
FROM poi_reunion_food
where category_2 = 'ActivityProvider'
LIMIT 5;

 * sqlite:///Itineraires.db


index,ID,Nom_du_POI,Latitude,Longitude,Adresse_postale,Date_de_mise_a_jour,Contacts_du_POI,Description,region,commune,code_postal,category_0,category_1,category_2
2,f2a140d5-152c-300b-accb-67619012a89a,Concorde (Le),-20.89625806,55.44723605,"91 bis, allée des Topazes - Bellepierre",2019-10-10,Concorde (Le)#+33 2 62 51 51 51#info@hotel-bellepierre.com#http://www.hotel-bellepierre.com/,None,54,Saint-Denis,97400,FoodEstablishment,Restaurant,ActivityProvider
35,4543e20f-0675-3254-9506-48b083a4a84a,El Pueblo,-21.343484,55.476775,"8, boulevard Hubert Delisle",2019-10-10,El Pueblo#+33 2 62 38 56 40##,None,54,Saint-Pierre,97410,FoodEstablishment,Restaurant,ActivityProvider
36,457271d5-8321-36b2-a1ba-20ea232b0483,Artocarpe,-20.876704,55.448779,"Résidence Edouard - 9, ruelle Edouard",2021-09-28,Artocarpe#+33 2 62 21 55 48#artocarpe@orange.fr#,None,54,Saint-Denis,97400,FoodEstablishment,Restaurant,ActivityProvider
54,316dd449-7195-3841-b20d-bd33fcdfb22f,Madiana Restaurant - Bar - Tapas,-21.343185,55.467639,83 Boulevard Hubert Delisle,2019-10-10,Madiana Restaurant - Bar - Tapas#+33 6 93 13 68 79##,None,54,Saint-Pierre,97410,FoodEstablishment,Restaurant,ActivityProvider
85,708d105f-f963-3c34-8f87-4bb3b0ea6c5a,Zanzibar Café (Le),-20.8786009,55.45154099,"41, rue Pasteur",2019-10-10,Zanzibar Café (Le)#+33 2 62 20 01 18##,None,54,Saint-Denis,97400,FoodEstablishment,Restaurant,ActivityProvider


In [143]:
%%sql
SELECT * FROM (
SELECT Nom_du_POI, Adresse_postale, "Activité1" 
FROM poi_reunion_2
WHERE commune = 'Saint-Denis' 
LIMIT 2)
UNION
SELECT * FROM (
SELECT Nom_du_POI, Adresse_postale, "Activité2" 
FROM poi_reunion_activity
where commune = 'Saint-Denis'
limit 2)
UNION
SELECT * FROM (
SELECT Nom_du_POI, Adresse_postale, "Restaurant" 
FROM poi_reunion_food
where commune = 'Saint-Denis'
limit 2)
ORDER BY 3;

 * sqlite:///Itineraires.db


Nom_du_POI,Adresse_postale,Activité1
Jardin de la Maison d'Edith (Le),"59, chemin Commins",Activité1
Musée Léon Dierx,"28, rue de Paris",Activité1
Varangue (La),"14, rue du Stade - Montgaillard",Activité2
Zen (Le),"2, rue Doret",Activité2
Concorde (Le),"91 bis, allée des Topazes - Bellepierre",Restaurant
Oasis (L'),"2, rue Doret",Restaurant
